In [ ]:

import regex as re
import pandas as pd
import numpy as np
from termcolor import colored
from urllib.request import urlopen
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

: 

In [ ]:
df = pd.read_csv(r'Dataset\calgary_cleaned_rentals_dataset.csv')

: 

In [ ]:
df.head(1)

: 

In [6]:
df[(~df.sq_feet.isna())&(df.sq_feet2.isna())].head(1)#['link'].values

,rented,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,sq_feet2,price2,beds2,baths2,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
2,Not-Rented,Immediate,2000-01-01,Calgary,Cliff Bungalow,51.036696,-114.074492,/ab/calgary/rentals/apartment/studio/cliff-bungalow/357976,Apartment,525.0,studio,420,1,0,0,NaN,NaN,0,0.0,1,1,1,0,0


In [35]:
df.dropna(subset=['price'], inplace=True)

In [11]:
import plotly.graph_objects as go

import pandas as pd

fig = go.Figure(data=go.Scattergeo(
        lon = df['longitude'],
        lat = df['latitude'],
        text = df['type'],
        mode = 'markers',
        #color = df['type'],
        ))

fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo_scope='north america',
    )
fig.show()

In [36]:
df.isna().sum()#[['price', 'price2']]

rented                    0
availability             67
a                         0
city                      1
community                 0
latitude                  0
longitude                 0
link                      0
type                      0
price                     0
beds                      0
sq_feet                 568
baths                     0
cats                      0
dogs                      0
sq_feet2               2404
price2                 2360
beds2                     0
baths2                    0
utility_heat              0
utility_electricity       0
utility_water             0
utility_cable             0
utility_internet          0
dtype: int64

In [32]:
len(df)

2774

### practice on data

In [ ]:
def get_json(url):
    url = url.replace(" ", "%20")
    response = urlopen(url)
    data_json = json.loads(response.read())
    return data_json

In [ ]:
num_beds = '' # a digit or ''
num_baths = ''
min_price = ''
max_price = ''
available_in = ''
city = 'calgary'
types = [
    'Apartment', 'Shared', 'Basement', 'Condo',
    'Loft', 'House', 'Main Floor', 'Townhouse'
    ]
# if the result was 500 lines, start searching about the range of prices

df = pd.DataFrame()
for type in types: 

    custom_url = f'https://www.rentfaster.ca/api/map.json?cities={city}&type={type}&price_range_adv[from]=null&price_range_adv[to]=null'
    
    data_json = get_json(custom_url)
    listings = data_json['listings']
    df_subset = pd.DataFrame(listings)
    df = pd.concat([df, df_subset], axis=0)

    # print('Unique ids = ', (df.id.unique()))
    print("Number of this section's unique ids = ", len(df.id.unique()))
    print('Number of all unique ids = ', len(df.id.unique()))

    if 'search' in data_json.keys(): print('My search = ', data_json['search'])
    if 'query' in data_json.keys(): print('My query = ', data_json['query'])

Number of this section's unique ids =  500
Number of all unique ids =  500
My search =  {'cities': 'calgary', 'type': ['Apartment'], 'price_range_adv': {'from': 'null', 'to': 'null'}}
Number of this section's unique ids =  728
Number of all unique ids =  728
My search =  {'cities': 'calgary', 'type': ['Shared'], 'price_range_adv': {'from': 'null', 'to': 'null'}}
Number of this section's unique ids =  1061
Number of all unique ids =  1061
My search =  {'cities': 'calgary', 'type': ['Basement'], 'price_range_adv': {'from': 'null', 'to': 'null'}}
Number of this section's unique ids =  1561
Number of all unique ids =  1561
My search =  {'cities': 'calgary', 'type': ['Condo'], 'price_range_adv': {'from': 'null', 'to': 'null'}}
Number of this section's unique ids =  1571
Number of all unique ids =  1571
My search =  {'cities': 'calgary', 'type': ['Loft'], 'price_range_adv': {'from': 'null', 'to': 'null'}}
Number of this section's unique ids =  1848
Number of all unique ids =  1848
My search 

In [ ]:
df_total = pd.DataFrame()
len(df_total)

0

In [ ]:
types = [
    'Apartment', 'Shared', 'Basement', 'Condo',
    'Loft', 'House', 'Main Floor', 'Townhouse'
    ]
price_ranges = [
    (100, 600), (600, 1000), (1000, 1500), (1500, 2000), (2000, 3000), (3000, 4000), (4000, 5000)
    ]
def scrape_data(city='calgary', types=types, price_ranges=price_ranges):
    df_total = pd.DataFrame()

    for type in types:

        url = f'https://www.rentfaster.ca/api/map.json?cities={city}&type={type}'
        
        # scrape based on residence type
        print('\nQuerying based on residence type ', colored(f'{type}', 'blue'))

        data_json = get_json(url)
        listings = data_json['listings']
        df_subset = pd.DataFrame(listings)

        len_subset_ids = len(df_subset.id.unique())
        print("Section unique ids = ", len_subset_ids)

        print('My search = ', data_json['search'])

        if len_subset_ids==500:
            print(colored('More than 500 records!', 'red'))

            for (s, e) in price_ranges:
                url = f'https://www.rentfaster.ca/api/map.json?cities={city}&type={type}&price_range_adv[from]={s}&price_range_adv[to]={e}'

                
                print('Querying based on range ', colored(f'{s} to {e}', 'green'))
                if len(df_total)!=0: print(' -- All unique ids = ', len(df_total.id.unique()))

                data_json = get_json(url)
                listings = data_json['listings']
                df_subset = pd.DataFrame(listings)
                df_total = pd.concat([df_total, df_subset], axis=0)
        else:
            df_total = pd.concat([df_total, df_subset], axis=0)
            print('All unique ids = ', len(df_total.id.unique()))

    return df_total
df = scrape_data(city='calgary', types=types)

In [ ]:
len(df)

2856

In [ ]:
df = scrape_data(city='calgary', types=types)


Querying based on residence type  Apartment
Section unique ids =  500
My search =  {'cities': 'calgary', 'type': ['Apartment']}
More than 500 records!
Querying based on range  100 to 600
Querying based on range  600 to 1000
 -- All unique ids =  4
Querying based on range  1000 to 1500
 -- All unique ids =  181
Querying based on range  1500 to 2000
 -- All unique ids =  548
Querying based on range  2000 to 3000
 -- All unique ids =  663
Querying based on range  3000 to 4000
 -- All unique ids =  681
Querying based on range  4000 to 5000
 -- All unique ids =  683

Querying based on residence type  Shared
Section unique ids =  229
All unique ids =  684
My search =  {'cities': 'calgary', 'type': ['Shared']}

Querying based on residence type  Basement
Section unique ids =  335
All unique ids =  913
My search =  {'cities': 'calgary', 'type': ['Basement']}

Querying based on residence type  Condo
Section unique ids =  500
All unique ids =  1246
My search =  {'cities': 'calgary', 'type': ['Co

In [ ]:
num_beds = '' # a digit or ''
num_baths = ''
min_price = ''
max_price = ''
available_in = ''
various_ranges = [
    (200,600), (600, 1000), (1000, 1500), (1500, 2000), (2000, 3000), (3000, 4000), (4000, 5000)
    ]
type = 'Apartment'
city = 'calgary'

df_apartment = pd.DataFrame()
for (s, e) in various_ranges: 

    custom_url = f'https://www.rentfaster.ca/api/map.json?cities={city}&type={type}&price_range_adv[from]={s}&price_range_adv[to]={e}'
    
    data_json = get_json(custom_url)
    listings = data_json['listings']
    df_subset = pd.DataFrame(listings)
    df_apartment = pd.concat([df_apartment, df_subset], axis=0)

    # print('Unique ids = ', (df.id.unique()))
    print("Number of this section's unique ids = ", len(df_subset.id.unique()))
    print('Number of all unique ids = ', len(df_apartment.id.unique()))

    if 'search' in data_json.keys(): print('My search = ', data_json['search'])
    if 'query' in data_json.keys(): print('My query = ', data_json['query'])

Number of this section's unique ids =  4
Number of all unique ids =  4
My search =  {'cities': 'calgary', 'type': ['Apartment'], 'price_range_adv': {'from': '200', 'to': '600'}}
Number of this section's unique ids =  182
Number of all unique ids =  186
My search =  {'cities': 'calgary', 'type': ['Apartment'], 'price_range_adv': {'from': '600', 'to': '1000'}}
Number of this section's unique ids =  454
Number of all unique ids =  560
My search =  {'cities': 'calgary', 'type': ['Apartment'], 'price_range_adv': {'from': '1000', 'to': '1500'}}
Number of this section's unique ids =  205
Number of all unique ids =  679
My search =  {'cities': 'calgary', 'type': ['Apartment'], 'price_range_adv': {'from': '1500', 'to': '2000'}}
Number of this section's unique ids =  70
Number of all unique ids =  698
My search =  {'cities': 'calgary', 'type': ['Apartment'], 'price_range_adv': {'from': '2000', 'to': '3000'}}
Number of this section's unique ids =  13
Number of all unique ids =  700
My search =  {

In [39]:
df_apartment.columns

Index(['ref_id', 'id', 'userId', 'rented', 'phone', 'phone_2', 'email',
       'availability', 'a', 'v', 'f', 's', 'intro', 'city', 'community',
       'latitude', 'longitude', 'marker', 'link', 'thumb2',
       'preferred_contact', 'type', 'price', 'beds', 'sq_feet', 'baths',
       'cats', 'dogs', 'utilities_included', 'title', 'sq_feet2', 'price2',
       'beds2', 'baths2'],
      dtype='object')

In [46]:
df.columns

Index(['ref_id', 'id', 'userId', 'rented', 'phone', 'phone_2', 'email',
       'availability', 'a', 'v', 'f', 's', 'title', 'intro', 'city',
       'community', 'latitude', 'longitude', 'marker', 'link', 'thumb2',
       'preferred_contact', 'type', 'price', 'price2', 'beds', 'beds2',
       'sq_feet', 'sq_feet2', 'baths', 'baths2', 'cats', 'dogs',
       'utilities_included'],
      dtype='object')

In [50]:
len(df_apartment)

928

In [51]:
len(df_concat)

3148

In [74]:
df = convert_utilities_col(df)

In [16]:
len(df.id.unique())

2276

In [73]:
def convert_utilities_col(df):
    df.loc[:, 'utility_heat'] = df.loc[:, 'utilities_included'].apply(lambda x: 1 if 'Heat' in x else 0)
    df.loc[:, 'utility_electricity'] = df.loc[:, 'utilities_included'].apply(lambda x: 1 if 'Electricity' in x else 0)
    df.loc[:, 'utility_water'] = df.loc[:, 'utilities_included'].apply(lambda x: 1 if 'Water' in x else 0)
    df.loc[:, 'utility_cable'] = df.loc[:, 'utilities_included'].apply(lambda x: 1 if 'Cable' in x else 0)
    df.loc[:, 'utility_internet'] = df.loc[:, 'utilities_included'].apply(lambda x: 1 if 'Internet' in x else 0)

    df.drop(columns=['utilities_included'], inplace=True)
    return df


In [21]:
df.drop_duplicates(inplace=True)

In [15]:
len(total_df.drop(columns=['utilities_included']).drop_duplicates())

2283

In [ ]:

cols_to_drop = ['phone', 'phone_2', 'f', 's', 'title',
                'intro', 'userId', 'id', 'ref_id',
                'v', 'thumb2','link', 'marker', 'preferred_contact'
                ]
                
def drop_unwanted_cols(df, cols=cols_to_drop):
    all_columns = df.columns
    for col in cols:
        if col in all_columns:
            df.drop(columns=[col], inplace=True)
    return df

In [26]:


df.drop(columns=cols_to_drop, inplace=True)

In [29]:
df = convert_utilities_col(df)
df.dropna(subset=['sq_feet', 'type', 'price'], inplace=True)
df.fillna(0, inplace=True)

In [27]:
df.isna().sum()


rented                    0
email                     0
availability              0
a                         0
title                  2247
city                      0
community                 0
latitude                  0
longitude                 0
type                      0
price                     0
price2                 2051
beds                      0
beds2                  2092
sq_feet                   0
sq_feet2               2110
baths                     0
baths2                 2203
cats                      0
dogs                      0
utility_heat              0
utility_electricity       0
utility_water             0
utility_cable             0
utility_internet          0
dtype: int64

In [39]:
len(df) == len(df.drop_duplicates())

False

In [42]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True)

In [44]:
df.head(1)

,index,ref_id,id,userId,rented,phone,phone_2,email,availability,a,v,f,s,eid,intro,city,community,latitude,longitude,marker,link,thumb2,preferred_contact,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
0,0,377368,377368,25428,Not-Rented,4035080684,,True,Immediate,2000-01-01,1,0,RSYNC,377368,3520 31 St. NW,Calgary,Brentwood,51.084169,-114.128718,00fb1aee1575c135383933289fff598f,/ab/calgary/rentals/apartment/1-bedroom/brentw...,https://f1a3d4fea3a9a877e732-356deb4d9644d2835...,Phone||Text||Email,Apartment,2305,none,527,1,2.0,2.0,1,1,1,0,0


In [45]:
not_int = []
for i in range(len(df)):
    try: int(df.loc[i, 'sq_feet'])
    except: not_int.append(i)

df.drop(not_int, inplace=True)
df.reset_index(inplace=True)

In [48]:
df.fillna(0, inplace=True)

In [ ]:
df.loc[:, 'price'] = df.loc[:, 'price'].apply(lambda x: int(x))

### Practice .1


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/hzjken/SG-rental-analytics/master/code_and_data/model_data.csv'
df = pd.read_csv(url)
df.head()

,area,bedroom,first_i_school_dist,first_mrt_dist,first_p_school_dist,price,psf,sec_i_school_dist,sec_mrt_dist,sec_p_school_dist,...,count,dist_to_shops,dist_to_foodcourt,dist_to_supermarket,dist_to_downtown,apartment,bungalow,condominium,hdb,terrace_house
0,3009,4,0.49,0.57,0.82,5300,1.76,3.86,1.21,1.39,...,1,1.428765,2.134875,0.985758,14.407814,0,0,0,0,1
1,3009,4,0.49,0.57,0.82,5300,1.76,3.86,1.21,1.39,...,1,1.428765,2.134875,0.985758,14.407814,0,0,0,0,1
2,970,2,1.54,0.54,0.80,4000,4.12,1.82,0.61,1.47,...,1,0.633214,0.136767,0.244170,1.716227,0,0,1,0,0
3,1970,2,1.54,0.54,0.80,5200,2.64,1.82,0.61,1.47,...,1,0.633214,0.136767,0.244170,1.716227,0,0,1,0,0
4,1044,2,1.29,0.82,0.37,5500,5.27,1.38,0.98,1.42,...,1,1.057692,0.475773,0.626818,2.198972,1,0,0,0,0


In [3]:
pd.read_json('https://www.rentfaster.ca/api/map.json?e=undefined&beds=&baths=&type=&price_range_adv%5Bfrom%5D=450price_range_adv%5Bfrom%5D=3000&availability=May')

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
price_range_adv%5Bto%5D=2300&price_range_adv%5Bfrom%5D=450

In [ ]:
from bs4 import BeautifulSoup
import urllib2
import csv

url = 'https://www.rentfaster.ca//ab/calgary'
html = urllib2.urlopen(url).read()
soup = BeautifulSoup(html)
table = soup.select_one("table.data2_s")
# python3 just use th.text
headers = [th.text.encode("utf-8") for th in table.select("tr th")]

with open("out.csv", "w") as f:
    wr = csv.writer(f)
    wr.writerow(headers)
    wr.writerows([[td.text.encode("utf-8") for td in row.find_all("td")] for row in table.select("tr + tr")])